# 14-day cumulative number of COVID-19 cases per 100 000 for districts (Landkreise) in Germany

At the end of the page, we provide a detailed description of how the numbers are calculated.

### Compute data

In [1]:
import datetime as dt
import pandas as pd
pd.set_option("max_rows", None)
from oscovida import fetch_data_germany, \
    germany_get_population, get_country_data

# get districts
germany = fetch_data_germany()
districts = sorted(germany['Landkreis'].drop_duplicates())

Please be patient - downloading data from https://www.arcgis.com/sharing/rest/content/items/f10774f1c63e40168479a1feb6c7ca74/data ...
Completed downloading 181903 rows in 11.0 seconds.


In [2]:
data = []
yesterday = dt.date.today() - dt.timedelta(days=1) 

for i, district in enumerate(districts):
    if i % 100 == 0:
        print(f"Processing {i}/412 ({district})")
    c, _, _ = get_country_data(country="Germany", 
                               subregion=district) 
    if c.index[-1].date() < yesterday:
        print(f"{district}: last data is from {c.index[-1].date()}")
        origin = c.index[0].date()
        # Fill data series forward up to yesterday
        new_idx = pd.date_range(origin, periods=(yesterday - origin).days, freq='D')
        c.reindex(new_idx, method='pad')
    c = c[-15:]

    population = germany_get_population(landkreis=district)
    new_cases = int(c[-1] - c[-15]) 
    incidence = new_cases / population * 100000. 
    data += [(district, population, new_cases, round(incidence, 1))]


Processing 0/412 (LK Ahrweiler)
Processing 100/412 (LK Hersfeld-Rotenburg)
Processing 200/412 (LK Regensburg)
Processing 300/412 (SK Berlin Charlottenburg-Wilmersdorf)
Processing 400/412 (SK Trier)


In [3]:
# sort, and ignore SK and LK for sorting
data.sort(key=lambda x: x[0].replace("SK ", "").replace("LK ", ""))

# turn into pandas DataFrame for easier display
table = pd.DataFrame(data, 
                     columns=["district", "population", 
                              "new cases", "14-day-incidence"]
                    ).set_index("district")

# Show last update date
import time
print(f"Last updated {time.asctime()}")

Last updated Tue Aug 18 18:06:09 2020


## Table for all districts (=Landkreise)

In [4]:
table

,population,new cases,14-day-incidence
district,,,
LK Ahrweiler,129727,28,21.6
LK Aichach-Friedberg,133596,4,3.0
LK Alb-Donau-Kreis,196047,22,11.2
LK Altenburger Land,90118,0,0.0
LK Altenkirchen,128705,10,7.8
LK Altmarkkreis Salzwedel,83765,1,1.2
LK Altötting,111210,39,35.1
LK Alzey-Worms,129244,18,13.9
SK Amberg,41970,4,9.5


## Table sorted by 14-day-incidence

In [5]:
table.sort_values(by="14-day-incidence", ascending=False)

,population,new cases,14-day-incidence
district,,,
LK Dingolfing-Landau,96217,206,214.1
SK Herne,156374,90,57.6
SK Duisburg,498590,270,54.2
SK Offenbach,128744,68,52.8
SK Berlin Mitte,384172,193,50.2
SK Wuppertal,354382,163,46.0
SK Bochum,364628,166,45.5
LK Unna,394782,172,43.6
SK Hagen,188814,82,43.4


## Tutorial: Detailed calculation for one district

In [6]:
cases, deaths, label = get_country_data("Germany", subregion="SK Hamburg")

Cumulative cases yesterday (numbers for today are not known yet):

In [7]:
c_y = cases[-1]
c_y

5901

Cumulative cases 15 days ago (i.e. 14 days before yesterday)

In [8]:
c_15 = cases[-15]
c_15

5490

New cases from the last 14 days

In [9]:
c_new = c_y - c_15 
c_new

411

Get the population data

In [10]:
population = germany_get_population(landkreis="SK Hamburg")
population

1841179

Compute the 14-day incidence per 100000, i.e. the cumulative number of new infections in the last 14 days, normalised by the country's population in units of 100000:

In [11]:
incidence = c_new / (population/100000)
incidence

22.322653039166752

---------------

In [12]:
import oscovida
oscovida.display_binder_link('14-day-incidence-germany.ipynb')

[Execute this notebook with Binder](https://mybinder.org/v2/gh/oscovida/binder/master?filepath=ipynb/14-day-incidence-germany.ipynb)